In [ ]:
from IPython.core.interactiveshell import InteractiveShell
import os
import sys
import time
from functools import partial
import pickle
import multiprocessing
import pixiedust as pxdb
import PIL
from matplotlib import pyplot as plt
import seaborn as sns
from collections import OrderedDict as ODict
import numpy as np
import pandas as pd

# PYTORCH
import torch as tc
import torchvision as tcvis
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torch.nn import functional as fu
# TORCH HELPERS
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
import hiddenlayer as hl
from torchsummary import summary

InteractiveShell.ast_node_interactivity = 'all'

# %pixie_debugger
%matplotlib inline

%config InlineBackend.figure_format = 'retina'
sns.set(style='white', context='notebook', palette='tab10')
%config IPCompleter.greedy = True
%config IPCompleter.use_jedi = True


basepath = '/home/alk/Documents/Git/Kaggles/MNIST'
try:
    os.chdir(basepath)
    print(os.getcwd())
except NotADirectoryError:
    pass

In [ ]:
datadir = os.getcwd() + '/data'
filenames = ['train.csv', 'test.csv']
datadict = ODict()
for files in filenames:
    try:
        with open(datadir + '/' + files, mode='r') as csvfile:
            datadict[files] = np.loadtxt(csvfile, delimiter=",", skiprows=1)
            csvfile.close()
        print('found file: {}'.format(files))
    except FileNotFoundError:
        print('skipping file ./{}'.format(files))

datadict.keys(), filenames

In [ ]:
traindata = datadict[filenames[0]]
testdata = datadict[filenames[-1]]

trainlabels = traindata[:, 0].reshape(-1, 1)
traindata = traindata[:, 1:].reshape(-1, 28, 28)
testdata = testdata.reshape(-1, 28, 28)
print(traindata.shape, trainlabels.shape, testdata.shape)

In [ ]:
fig, ax = plt.subplots(1, 2, sharex=True, squeeze=True)
ax[0].imshow(traindata[-1, :, :], cmap='gray')
ax[1].imshow(testdata[0, :, :], cmap='gray')

In [ ]:
class NpDataset(Dataset):

    def __init__(self, x=traindata, y=trainlabels,
                 transforms=None):
        super().__init__()

        self.x = x
        self.y = y
        self.transform = transforms
        
    def __len__(self):

        return self.x.shape[0]

    def __getitem__(self, index):

        if self.y is not None:
            image, label = self.x[index], self.y[index]
            label = tc.from_numpy(label).type(tc.LongTensor)
        else:
            image, label = self.x[index], None
        
        # HxWxC, UINT8
        image = image.astype(np.uint8).reshape(28, 28, 1)

        if self.transform is not None:
            image = self.transform(image)

        return image, label

# test
test_set = NpDataset()
print(f'target: {test_set.__getitem__(0)[1]}')
plt.imshow(test_set.__getitem__(0)[0].reshape(28, 28), cmap='gray')

In [ ]:
MNIST_train_transform = transforms.Compose([transforms.ToPILImage(),
                                            transforms.RandomVerticalFlip(0.25),
                                            transforms.RandomAffine(30, (0.15,0.15),
                                                                    (0.75,1.25), 30,
                                                                    PIL.Image.BICUBIC,0),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.1307,),
                                                                 (0.3081,))])

MNIST_test_transform = transforms.Compose([transforms.ToTensor(),
                                           transforms.Normalize((0.1307,),
                                                                (0.3081,))])

# test
test_set = NpDataset(transforms=MNIST_train_transform)

fig, axes = plt.subplots(5, 5, sharex=True, sharey=True, squeeze=True)
_plots = None

for axs in axes:
    for ax in axs:
        _plots = ax.imshow(test_set.__getitem__(np.random.randint(0, \
                 traindata.shape[0]))[0].numpy().reshape(28, 28),
                 cmap='gray');

_plots = plt.yticks([], [])
_plots = plt.xticks([], [])
_plots = plt.axis('off')
plt.show()

In [ ]:
NN_trainloader = DataLoader(NpDataset(transforms=MNIST_train_transform),
                            batch_size=128,
                            shuffle=True,
                            num_workers=8,
                            pin_memory=True)

NN_testloader = DataLoader(NpDataset(x=testdata, y=None,
                                     transforms=MNIST_test_transform),
                            batch_size=128,
                            shuffle=True,
                            num_workers=8,
                            pin_memory=True)

In [ ]:
class MNIST_CNN(nn.Module):
    
    def __init__(self,
                 dropout_fcp=0.0,
                 wcuda=True):
        super(MNIST_CNN, self).__init__()
        
        self.dropout_p = dropout_fcp
        self.wcuda = wcuda
        
        self.conv1a = nn.Conv2d(1, 20, 5, 1) #24
        self.conv2a = nn.Conv2d(20, 50, 5, 1) #20
        
        self.conv1b = nn.Conv2d(1, 20, 5, 1)
        self.conv2b = nn.Conv2d(20, 50, 5, 1)
        
        self.fc1a = nn.Linear(4*4*50, 500)
        self.fc1b = nn.Linear(5*5*50, 500)
        
        self.fcf1 = nn.Linear(1000,100)
        self.fcf2 = nn.Linear(100,10)
        
    def forward(self, x):
        
        x1 = fu.relu(self.conv1a(x))
        x1 = fu.max_pool2d(x1, 2, 2)
        x1 = fu.relu(self.conv2a(x1))
        x1 = fu.max_pool2d(x1, 2, 2)
        x1 = x1.view(-1, 4*4*50)
        x1 = fu.relu(self.fc1a(x1))
        x1 = fu.dropout(x1, p=self.dropout_p, 
                        training=self.training)
        
        x2 = fu.relu(self.conv1a(x))
        x2 = fu.relu(self.conv2a(x2))
        x2 = fu.max_pool2d(x2, 4, 4)
        x2 = x2.view(-1, 5*5*50)
        x2 = fu.relu(self.fc1b(x2))
        x2 = fu.dropout(x2, p=self.dropout_p, 
                        training=self.training)
        
        x = tc.cat((x1, x2), 1)
        
        x = fu.relu(self.fcf1(x))
        x = fu.dropout(x, p=self.dropout_p,
                       training=self.training)
        
        x = self.fcf2(x)
        x = fu.log_softmax(x, dim=1)
        
        if self.wcuda:
            return x.cuda()
        else:
            return x
        
# test

test_cnn = MNIST_CNN().cuda()
print(summary(test_cnn.cuda(), (1,28,28), device='cuda'))
hl.build_graph(model=test_cnn.cuda(), args=tc.randn(1,1,28,28).cuda())

In [ ]:
 def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        #output = tc.argmax(output, 1).view(-1,1).type(tc.float16)
        #print(output.shape, target.shape)
        #print(output.type(), target.type())
        loss = fu.nll_loss(output, target.view(-1))
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += fu.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [ ]:
MNISTmodel = MNIST_CNN(dropout_fcp=0.25)
optimizer = optim.Adam(MNISTmodel.parameters(), lr=0.001)

device = tc.device('cuda')
epochs = 100

for epoch in range(1, epochs+1):
    train(MNISTmodel, device, NN_trainloader, optimizer, epoch, 250)